In [4]:
import numpy as np
from numba import njit
import math
from geo3d import norm_L2, normalized_vector, normalized_quat, quat_as_matrix, UnitFrame, Vector, Point, cast_vec_to_array, Frame
from typing import Tuple

vec = [1,2,3,4]
vec_tup = tuple(vec)
vec_np = np.array(vec)
vec_geo = Vector(vec)
mat_np = np.eye(4)

## Frame from quaternion speedup

In [5]:
%%timeit
Frame._from_quat_and_translations_scipy(1,2,3,4,1,2,3)

64.8 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%%timeit
quat_as_matrix(vec_tup)

598 ns ± 6.09 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%%timeit
Frame.from_quat_and_translations(1,2,3,4,1,2,3)

4.2 µs ± 82.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Normalization

In [2]:
def norm_np(vec):
    return np.linalg.norm(vec)

def norm_py(vec):
    return sum(v**2 for v in vec)

@njit
def norm_jit_np(vec):
    return np.sum(vec**2)

@njit
def norm_L2_new(vec):
    s = 0
    for v in vec:
        s += v**2
    return math.sqrt(s)

@njit
def normalized_vector_array(vec) -> np.ndarray:
    """ Return unit vector
    
    by dividing by Euclidean (L2) norm
    
    Args:
        vec array with elements x,y,z
    
    Returns: 
        array shape (3,) vector divided by L2 norm
    """
    res = np.empty(3)
    n = norm_L2(vec)
    res[0] = vec[0] / n
    res[1] = vec[1] / n
    res[2] = vec[2] / n
    return res

@njit
def normalized_vector_tuple(vec) -> Tuple[float, float, float]:
    """ Return unit vector
    
    by dividing by Euclidean (L2) norm
    
    Args:
        vec array with elements x,y,z
    
    Returns: 
        array shape (3,) vector divided by L2 norm
    """
    n = norm_L2(vec)
    return (vec[0] / n, vec[1] / n, vec[2] / n)


@njit
def normalized_quat_new(quat):
    n = norm_L2(vec)
    nq = (quat[0]/n, quat[1]/n, quat[2]/n, quat[3]/n)
    return nq

@njit
def cast_vec_to_array(vec):
    a = np.empty(3)
    a[0] = vec[0]
    a[1] = vec[1]
    a[2] = vec[2]
    return a

In [3]:
np.empty(3, dtype=float)

array([0., 0., 0.])

In [5]:
%%timeit
norm_np(vec)

7.08 µs ± 491 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [6]:
%%timeit
norm_py(vec)

1.81 µs ± 8.56 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
%%timeit
norm_jit_np(np.array(vec))

1.38 µs ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%%timeit
normalized_vector_array(vec_tup)

606 ns ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%timeit
np.array(normalized_vector(vec_tup))

1.12 µs ± 8.17 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
%%timeit
vec_geo.normalize()

1.35 µs ± 96.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
%%timeit
vec_np/norm_np(vec)

8.93 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### methods in geo3d

In [16]:
%%timeit
norm_L2(vec_tup)

319 ns ± 9.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
%%timeit
normalized_vector(vec_tup)

669 ns ± 3.58 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## vectors manipulation

In [21]:
@njit
def add_vectors(v1,v2):
    return (v1[0]+v2[0], v1[1]+v2[1], v1[2]+v2[2])

@njit
def mult_vector_scalar(v,s):
    return (v[0]*s, v[1]*s, v[2]*s)

@njit
def dot_vectors(v1,v2):
    return v1[0]*v2[0] + v1[1]*v2[1] + v1[2]*v2[2]

@njit
def mult_mat_vec(m,v):
    return (
        dot_vectors(m[0], v),
        dot_vectors(m[1], v),
        dot_vectors(m[2], v)
    )

In [22]:
%%timeit
add_vectors(vec_tup, vec_tup)

426 ns ± 3.85 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [23]:
%%timeit
vec_np+vec_np

472 ns ± 21.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [24]:
%%timeit
mat_np@vec_np

1.64 µs ± 7.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [32]:
%%timeit
mult_vector_scalar(vec_np, 3)

320 ns ± 4.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [36]:
%%timeit
vec_np * 3

735 ns ± 8.17 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [40]:
%%timeit
dot_vectors(vec_np, vec_np)

403 ns ± 5.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [42]:
%%timeit
mult_mat_vec(mat_np, vec_np)

429 ns ± 3.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [25]:
%%timeit
mat_np @ vec_np

1.68 µs ± 42.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [28]:
%%timeit
Point.from_array(
    mat_np @ vec_np + vec_np, copy=False
)

3.06 µs ± 21.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [38]:
%%timeit
Point(add_vectors(mult_mat_vec(mat_np, vec_np), vec_np))

2.17 µs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
@njit
def trafo(rot, trans, p):
    return cast_vec_to_array(add_vectors(mult_mat_vec(rot, p), trans))

In [42]:
%%timeit
Point.from_array(trafo(mat_np, vec_np, vec_np), copy=False)

1.27 µs ± 5.27 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Casting to array

In [43]:
%%timeit
np.array(vec)

933 ns ± 8.65 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [44]:
%%timeit
np.array(vec_np)

426 ns ± 2.19 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [45]:
%%timeit
np.asarray(vec_np)

298 ns ± 0.959 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [46]:
%%timeit
Vector.from_array(vec_np, copy=False)

346 ns ± 2.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [47]:
%%timeit
Vector(vec_np)

677 ns ± 4.44 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [49]:
%%timeit
cast_vec_to_array(vec_np)

720 ns ± 80 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
